In [1]:
from generator import LowShotGenerator
from classifier import Classifier
from callbacks import CloudCallback
from train import get_trained_classifier_and_data

import numpy as np
import collect


n_clusters = 20
n_files = 12

all_diseases = ['Atelectasis',
                'Cardiomegaly',
                'Consolidation',
                'Edema',
                'Effusion',
                'Emphysema',
                'Fibrosis',
                'Hernia',
                'Infiltration',
                'Mass',
                'No Finding',
                'Nodule',
                'Pleural_Thickening',
                'Pneumonia',
                'Pneumothorax']

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
unused_diseases = ['Hernia', 'Pneumonia', 'Edema', 'Emphysema', 'Fibrosis']
diseases_to_remove = list(unused_diseases)
diseases = [d for d in all_diseases if d not in unused_diseases]

quadruplets_data = collect.load_quadruplets(n_clusters=n_clusters, categories=diseases, n_files=n_files)

print('Unused diseases: {0}'.format(', '.join(unused_diseases)))

Loaded centroids from file
Unused diseases: Hernia, Pneumonia, Edema, Emphysema, Fibrosis


In [3]:
classifier, X_train, X_test, y_train, y_test = get_trained_classifier_and_data(diseases_to_remove, n_files=n_files)
classifier.toggle_trainability()  # make the classifier non-trainable

Loaded classifier and data from files
Loaded classifier weights from a saved model
Classifier is now non-trainable!


In [4]:
λ = 10.
lsg_name = 'lsg_f.{0}_c.{1}_w.{2}'.format(n_files, n_clusters, '.'.join(unused_diseases))
lsg = LowShotGenerator(classifier.model, quadruplets_data, λ=λ, name=lsg_name)
lsg.fit()

Generator summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 6144)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               3146240   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 2048)              1050624   
Total params: 4,459,520
Trainable params: 4,459,520
Non-trainable params: 0
_________________________________________________________________
None

Whole model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 6144)              0         
____________________

In [7]:
unused_data = collect.load_quadruplets(n_clusters=n_clusters, categories=unused_diseases, n_files=n_files)
quadruplets, centroids, cat_to_vectors, cat_to_onehots, original_shape = unused_data

n_examples = min(len(vecs) for cat, vecs in cat_to_vectors.items() if cat not in diseases_to_remove)
n_samples = 10
disease_to_hillucinate = disease = 'Hernia'

print('Will generate {0} examples from {1} samples of {2}'.format(n_examples, n_samples, disease_to_hillucinate))

Loaded centroids from file
Will generate 1093 examples from 10 samples of Hernia


In [8]:
X_train_disease, X_test_disease = cat_to_vectors[disease][:n_samples], cat_to_vectors[disease][n_samples:]
X_train_disease = np.concatenate([X_train_disease] + [lsg.generate(ϕ, n_new=n_examples // n_samples) for ϕ in X_train_disease])
y_train_disease = np.array([cat_to_onehots[disease] for x in X_train_disease])
y_test_disease = np.array([cat_to_onehots[disease] for x in X_test_disease])

X_train, y_train = np.concatenate((X_train, X_train_disease)), np.concatenate((y_train, y_train_disease))

In [9]:
fresh_classifier = Classifier(trainable=True)
fresh_classifier.fit(X_train, y_train)

Train on 72792 samples, validate on 8088 samples
Epoch 1/10
 - 5s - loss: 3.3911 - acc: 0.6289 - val_loss: 3.3825 - val_acc: 0.5749
Epoch 2/10
 - 5s - loss: 3.5073 - acc: 0.6410 - val_loss: 3.8389 - val_acc: 0.5739
Epoch 3/10
 - 5s - loss: 3.8305 - acc: 0.6407 - val_loss: 3.8315 - val_acc: 0.5679
Epoch 4/10
 - 5s - loss: 3.9047 - acc: 0.6442 - val_loss: 3.8363 - val_acc: 0.5779
Epoch 5/10
 - 5s - loss: 4.9941 - acc: 0.6776 - val_loss: 5.0452 - val_acc: 0.5817
Epoch 6/10
 - 5s - loss: 5.1249 - acc: 0.6820 - val_loss: 5.0452 - val_acc: 0.5817
Epoch 7/10
 - 5s - loss: 5.1249 - acc: 0.6820 - val_loss: 5.0452 - val_acc: 0.5817
Epoch 8/10
 - 5s - loss: 5.1249 - acc: 0.6820 - val_loss: 5.0452 - val_acc: 0.5817
Epoch 9/10
 - 5s - loss: 5.1249 - acc: 0.6820 - val_loss: 5.0452 - val_acc: 0.5817
Epoch 10/10
 - 5s - loss: 5.1249 - acc: 0.6820 - val_loss: 5.0452 - val_acc: 0.5817


In [10]:
loss, acc = classifier.evaluate(X_test, y_test)
print('accuracy for regular diseases is {0}'.format(acc))

loss, acc = classifier.evaluate(X_test_disease, y_test_disease)
print('accuracy for novel disease "{0}" is {1}'.format(disease, acc))

8865/8865 [==============================] - 0s 44us/step
accuracy for regular diseases is 0.677834179357022
100/100 [==============================] - 0s 57us/step
accuracy for novel disease "Hernia" is 0.0
